In [ ]:
import add_packages
import os, dotenv, yaml, requests
from pprint import pprint

from my_langchain import (
  prompts, runnables,
)

from langserve import RemoteRunnable

with open("./basic/config.yaml") as f:
  config = yaml.safe_load(f)

In [ ]:
model_openai = RemoteRunnable(f'{config["base_endpoint"]}/openai/')
model_anthropic = RemoteRunnable(f'{config["base_endpoint"]}/anthropic/')
chain_joke = RemoteRunnable(f'{config["base_endpoint"]}/joke/')


In [ ]:
msg = [
  prompts.SystemMessage(content="Act like King Clown."),
  prompts.HumanMessage(content="Tell me a joke about cat!")
]

async def run_astream(runnable: runnables.Runnable, msg):
  result = ""
  async for chunk in runnable.astream(msg):
    result += chunk.content
    print(chunk.content, end="", flush=True)
  
  print()
  return result
    
result = await run_astream(runnable=model_anthropic, msg=msg)
pprint(result)

In [ ]:
msg = {"topic": "dog"}

result = await run_astream(runnable=chain_joke, msg=msg)
pprint(result)

In [ ]:
# Using requests
response = requests.post(
  f'{config["base_endpoint"]}/joke/invoke',
  json={"input": {"topic": "cats"}},
)
response.json()

In [ ]:
async def invoke_chain(chain: runnables.Runnable, input, is_async: bool = False):
  if is_async:
    await chain.ainvoke(input)

  chain.invoke(input)

invoke_chain(chain_joke, input={"topic": "parrots"}, is_async=False)
await invoke_chain(chain_joke, input={"topic": "parrots"}, is_async=True)